In [1]:
import math
from os import path, walk
from json import dumps, loads
from pyvis.network import Network
from dataclasses import dataclass, field

from attack import Attack
from defend import DTechnique, DSystem, Defend
from sigma import Sigma, SigmaRule
from types import SimpleNamespace

In [2]:
d = Defend(Attack(local=True))
s = Sigma()

gconf = SimpleNamespace(**d.attack.cfg.graph)

[]

In [105]:
Out = './Poligon'

net = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')

In [106]:
# Tactics
prev = None
for tac in d.attack.cfg.tactics_order:
    net.add_node(tac, phyisics=False,label=tac.replace('-', ' ').title(), **gconf.tactic)
    if prev:
        net.add_edge(prev, tac)
        pass
    prev = tac

In [107]:
for tech in d.attack.techniques:
    tac = d.attack.techniques[tech].tactics[0]
    
    parent = d.attack.techniques[tech].get_parent()
    
    if parent and parent not in net.node_ids:
        parent_tac = d.attack.techniques[parent].tactics[0]
        net.add_node(parent, label= d.attack.get_name_by_id(parent), **gconf.technique)
        #net.add_node(parent, color=gconf.technique['color'], mass=gconf.technique['mass'])
        net.add_edge(parent, parent_tac)
    
    if parent:
        net.add_node(tech, label = d.attack.get_name_by_id(tech),  **gconf.subtechnique)
        #net.add_node(tech, color=gconf.subtechnique['color'], mass=gconf.subtechnique['mass'])
        net.add_edge(tech, parent)
    else:
        net.add_node(tech, label = d.attack.get_name_by_id(tech), **gconf.technique)
        #net.add_node(tech, color=gconf.technique['color'], mass=gconf.technique['mass'])
        net.add_edge(tech, tac)

In [49]:
d.attack.techniques['T1542'].tactics

['defense-evasion', 'persistence']

In [109]:
#net.show_buttons()

netcfg = """{
  "edges": {
    "color": {
      "inherit": true
    },
    "selfReference": {
      "angle": 0.7853981633974483
    },
    "smooth": {
      "forceDirection": "none"
    }
  },
  "physics": {
    "barnesHut": {
      "centralGravity": 0.8,
      "springLength": 30,
      "damping": 0.21,
      "avoidOverlap": 0.13
    },
    "minVelocity": 0.75,
    "timestep": 1
  }
}"""
net.set_options(netcfg)
net.show('sigma_multinode_3_sev_Crit.html')

sigma_multinode_3_sev_Crit.html


In [189]:

net2 = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')
net2.add_node('A', physics=False)

net2.add_node('B', physics=False)
net2.add_node('B1', physics=True)
net2.add_node('B2', physics=True)
net2.add_node('C', physics=False)
net2.add_node('C1', physics=True)
net2.add_node('C2', physics=True)
net2.add_node('D', physics=False)
net2.add_edge('A', 'B', physics=False)
net2.add_edge('C1', 'A', physics=True)
net2.add_edge('C1', 'B', physics=True)
net2.add_edge('C1', 'D', physics=True)
net2.add_edge('D', 'C2', physics=True)

index = 0
result =  get_coords(radius=50000, x=0, y=0, minresults=15)
for pos in result:
    net2.add_node(index, x=pos[0], y=pos[1], physics=False)
    index +=1
    
    level2_result = get_coords(radius=3000, x = pos[0], y=pos[1], minresults=12)
    #print(level2_result)
    index2 = 0
    for pos2 in level2_result:
        net2.add_node(str(index)+"-"+str(index2), x=pos2[0], y=pos2[1], physics=False)
        index2 += 1
    
    
net2.show('test.html')

test.html


In [204]:
net3 = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')

# Tactics
prev = None

tactic_coords = get_coords(20000, x=0, y=0, minresults=len(d.attack.cfg.tactics_order)+1)
print(len(tactic_coords))
print(len(d.attack.cfg.tactics_order))
tac_index = 0

for tac in d.attack.cfg.tactics_order:
    net3.add_node(tac, physics=False, label=tac.replace('-', ' ').title(), x=tactic_coords[tac_index][0], y=tactic_coords[tac_index][1], **gconf.tactic)
    tac_index += 1
    if prev:
        net3.add_edge(prev, tac)
        pass
    prev = tac
    
# Techniques

for tac in full_index:
    


# for tech in d.attack.techniques:
#     tac = d.attack.techniques[tech].tactics[0]
    
#     parent = d.attack.techniques[tech].get_parent()
    
#     if parent and parent not in net3.node_ids:
#         parent_tac = d.attack.techniques[parent].tactics[0]
#         net3.add_node(parent, label= d.attack.get_name_by_id(parent), physics=False, **gconf.technique)
#         #net.add_node(parent, color=gconf.technique['color'], mass=gconf.technique['mass'])
#         net3.add_edge(parent, parent_tac)
    
#     if parent:
#         net3.add_node(tech, label = d.attack.get_name_by_id(tech),  **gconf.subtechnique)
#         #net.add_node(tech, color=gconf.subtechnique['color'], mass=gconf.subtechnique['mass'])
#         net3.add_edge(tech, parent)
#     else:
#         net3.add_node(tech, label = d.attack.get_name_by_id(tech),physics=False, **gconf.technique)
#         #net.add_node(tech, color=gconf.technique['color'], mass=gconf.technique['mass'])
#         net3.add_edge(tech, tac)

    
net3.show('test2.html')

15
14
{'execution': 15, 'collection': 16, 'persistence': 14, 'credential-access': 16, 'discovery': 28, 'resource-development': 8, 'reconnaissance': 10, 'defense-evasion': 42, 'impact': 13, 'lateral-movement': 9, 'command-and-control': 17, 'initial-access': 6, 'exfiltration': 9, 'privilege-escalation': 4}
test2.html


In [8]:
d.attack.cfg.graph

{'tactic': {'mass': 100, 'size': 100, 'color': '#C00'},
 'technique': {'mass': 50, 'size': 50, 'color': '#C77'},
 'subtechnique': {'mass': 5, 'size': 20, 'color': '#A66'},
 'severity_colors': {'critical': '#E76EFA',
  'high': '#AB62DE',
  'medium': '#9E77F5',
  'low': '#6762DE',
  'informational': '#FFF'}}

In [29]:
net4 = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')


tac_rad = 18000
tech_rad = 2500
subtech_rad = 300

# Tactics
prev = None
tactic_coords = d.get_coords(tac_rad, x=0, y=0, minresults=len(d.attack.cfg.tactics_order)+1)

tac_index = 0

for tac in d.attack.hierarchical_index:
    tac_x = tactic_coords[tac_index][0]
    tac_y = tactic_coords[tac_index][1]
    
    net4.add_node(tac, physics=False, label=tac.replace('-', ' ').title(), x=tac_x, y=tac_y, **d.attack.cfg.graph['tactic'])
    tac_index += 1
    if prev:
        net4.add_edge(prev, tac)
    prev = tac
    
    # Techniques
    tech_index = 0
    technique_coords = d.get_coords(tech_rad, x=tac_x, y=tac_y, minresults=len(d.attack.hierarchical_index[tac])+1)
    for tech in d.attack.hierarchical_index[tac]:
        tech_x = technique_coords[tech_index][0]
        tech_y = technique_coords[tech_index][1]
        net4.add_node(tech, physics=False, label= d.attack.get_name_by_id(tech), x=tech_x, y=tech_y, **d.attack.cfg.graph['technique'])
        tech_index += 1
        net4.add_edge(tech, tac)
        
        
        # Subtechniques
        subtech_index = 0
        subtechnique_coords = d.get_coords(subtech_rad, x=tech_x, y=tech_y, minresults=len(d.attack.hierarchical_index[tac][tech])+1)
        for subtech in d.attack.hierarchical_index[tac][tech]:
            subtech_x = subtechnique_coords[subtech_index][0]
            subtech_y = subtechnique_coords[subtech_index][1]
            net4.add_node(subtech, physics=False, label= d.attack.get_name_by_id(subtech), x=subtech_x, y=subtech_y, **d.attack.cfg.graph['subtechnique'])
            subtech_index += 1
            net4.add_edge(subtech, tech)

tactic_circles = {}            
for tac in d.attack.hierarchical_index:
    tactic_circles.update({
        tac: d.get_coords(tech_rad+500, x=net4.get_node(tac)['x'], y=net4.get_node(tac)['y'], minresults=100)
    })
    tactic_circles[tac].extend(d.get_coords(tech_rad+800, x=net4.get_node(tac)['x'], y=net4.get_node(tac)['y'], minresults=150))
    tactic_circles[tac].extend(d.get_coords(tech_rad+1100, x=net4.get_node(tac)['x'], y=net4.get_node(tac)['y'], minresults=200))
    tactic_circles[tac].extend(d.get_coords(tech_rad+1400, x=net4.get_node(tac)['x'], y=net4.get_node(tac)['y'], minresults=250))
    tactic_circles[tac].extend(d.get_coords(tech_rad+1700, x=net4.get_node(tac)['x'], y=net4.get_node(tac)['y'], minresults=300))
            
        


In [ ]:
taken = []
for rule in s.rules:
    for tech in rule.techniques:
        tac = d.attack.find_tactic_in_hierarchical_index(tech)
        
        tech_pos = (net4.get_node(tech)['x'],net4.get_node(tech)['y'])
        circle = tactic_circles[tac]
        closest_pos = d.find_closest_point(tech_pos, circle, taken)
        if closest_pos:
            net4.add_node(rule.id+tech, label=rule.title, color=d.attack.cfg.graph['severity_colors'][rule.severity], physics=False, x=closest_pos[0], y=closest_pos[1])
            net4.add_edge(rule.id+tech, tech)
            taken.append(closest_pos)
            
#net4.show('test4.html')

In [11]:
taken = []
for rule in s.rules:
    for tech in rule.techniques:
        tac = d.attack.find_tactic_in_hierarchical_index(tech)
        
        tech_pos = (net4.get_node(tech)['x'],net4.get_node(tech)['y'])
        circle = tactic_circles[tac]
        closest_pos = find_closest_point(tech_pos, circle, taken)
        if closest_pos:
            net4.add_node(rule.id+tech, label=rule.title, color=d.attack.cfg.graph['severity_colors'][rule.severity], physics=False, x=closest_pos[0], y=closest_pos[1])
            net4.add_edge(rule.id+tech, tech)
            taken.append(closest_pos)
            
#net4.show('test4.html')

test4.html


In [9]:
def draw_circular_attack_graph(net, tactic_radius, tech_radius, subtech_radius, connect_tactics = False):
    
    tactic_coords = d.get_coords(tactic_radius, 
                                 x = 0, y = 0, 
                                 minresults = len(d.attack.cfg.tactics_order)+1)
    
    prev = None
    tac_index = 0
    
    for tac in d.attack.hierarchical_index:
        tac_x = tactic_coords[tac_index][0]
        tac_y = tactic_coords[tac_index][1]

        net.add_node(tac, 
                     physics = False, 
                     label = tac.replace('-', ' ').title(), 
                     x = tac_x, y = tac_y, 
                     **d.attack.cfg.graph['tactic'])
        
        # Replace previous node
        tac_index += 1
        if prev:
            net.add_edge(prev, tac)
        prev = tac
    
        # Techniques
        tech_index = 0
        technique_coords = d.get_coords(tech_radius, 
                                        x=tac_x, y=tac_y, 
                                        minresults=len(d.attack.hierarchical_index[tac])+1)
        
        for tech in d.attack.hierarchical_index[tac]:
            tech_x = technique_coords[tech_index][0]
            tech_y = technique_coords[tech_index][1]
            net.add_node(tech, 
                         physics = False, 
                         label = d.attack.get_name_by_id(tech), 
                         x=tech_x, y=tech_y, 
                         **d.attack.cfg.graph['technique'])
            
            tech_index += 1
            net.add_edge(tech, tac, color='#333')


            # Subtechniques
            subtech_index = 0
            subtechnique_coords = d.get_coords(subtech_radius, 
                                               x = tech_x, y = tech_y, 
                                               minresults = len(d.attack.hierarchical_index[tac][tech])+1)
            
            for subtech in d.attack.hierarchical_index[tac][tech]:
                subtech_x = subtechnique_coords[subtech_index][0]
                subtech_y = subtechnique_coords[subtech_index][1]
                net.add_node(subtech, 
                             physics = True, 
                             label = d.attack.get_name_by_id(subtech), 
                             x = subtech_x, 
                             y = subtech_y, 
                             **d.attack.cfg.graph['subtechnique'])
                
                subtech_index += 1
                net.add_edge(subtech, tech)
    return net
    
def generate_defense_circles(net, radius, number_of_defense_circles, nodes_per_circle, increment_per_circle, circle_distance):
    circles = number_of_defense_circles
    node_count = nodes_per_circle
    increment = increment_per_circle
    dist = circle_distance
    
    tactic_circles = {}            
    for tac in d.attack.hierarchical_index: # remove this to rely on single while loop update({tac: []})
        tactic_circles.update({
            tac: d.get_coords(radius, 
                              x = net.get_node(tac)['x'], 
                              y = net.get_node(tac)['y'], 
                              minresults = node_count)
        })
        
        tmp_node_count = node_count + increment
        tmp_circles = circles
        tmp_radius = radius + dist
        while tmp_circles  > 0:
            tactic_circles[tac].extend(
                d.get_coords(tmp_radius+dist, 
                             x=net.get_node(tac)['x'], 
                             y=net.get_node(tac)['y'], 
                             minresults=node_count)
            )
            node_count += increment_per_circle
            tmp_radius += dist
            tmp_circles -= 1
    
    return tactic_circles

In [6]:
d.ge


In [13]:
net5 = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')


tac_rad = 18000
tech_rad = 2500
subtech_rad = 300

# Tactics
prev = None
tactic_coords = [
    (0,0), (0,1000)
]

tac_index = 0

net5.add_node('initial-access', physics=False, label=tac.replace('-', ' ').title(), x=tactic_coords[0][0], y=tactic_coords[0][1], **d.attack.cfg.graph['tactic'])
net5.add_node('execution', physics=False, label=tac.replace('-', ' ').title(), x=tactic_coords[1][0], y=tactic_coords[1][1], **d.attack.cfg.graph['tactic'])

net5.show('test5.html')

test5.html


In [16]:
net5 = Network(notebook=True, height='2000px', width='2200px', bgcolor='#000', select_menu=False, filter_menu=False, font_color='#FFF')

tac_rad = 18000
tech_rad = 3500
subtech_rad = 300

index = 0
coords = d.get_tactics_line_coordinates(distance=10000)

label_index = {
    "reconnaissance": "RECON",
    "resource-development": "RDEV" ,
    "initial-access": "INIT",
    "execution": "EXEC",
    "persistence": "PERS",
    "privilege-escalation": "PESC",
    "defense-evasion": "EVAS",
    "credential-access": "CRED",
    "discovery": "DISC",
    "lateral-movement": "LATMO",
    "collection": "COLL",
    "command-and-control": "C&C",
    "exfiltration": "EXFIL",
    "impact": "IMP"
}


for tac in d.attack.hierarchical_index:
    tac_x = coords[index][0]
    tac_y = coords[index][1]
    
    net5.add_node(tac, physics=False, label=tac.replace('-', ' ').title(), x=tac_x, y=tac_y, **d.attack.cfg.graph['tactic'])
    index+=1
    
    node_data = net5.get_node(tac)
    node_data.update({"font": {"size": 500, "color": "#CCC"}})
    
    # Techniques
    tech_index = 0
    technique_coords = d.get_coords(tech_rad, x=tac_x, y=tac_y, minresults=len(d.attack.hierarchical_index[tac])+1)
    for tech in d.attack.hierarchical_index[tac]:
        tech_x = technique_coords[tech_index][0]
        tech_y = technique_coords[tech_index][1]
        net5.add_node(tech, physics=False, label= d.attack.get_name_by_id(tech), x=tech_x, y=tech_y, **d.attack.cfg.graph['technique'])
        tech_index += 1
        net5.add_edge(tech, tac)
        
        
        # Subtechniques
        subtech_index = 0
        subtechnique_coords = d.get_coords(subtech_rad, x=tech_x, y=tech_y, minresults=len(d.attack.hierarchical_index[tac][tech])+1)
        for subtech in d.attack.hierarchical_index[tac][tech]:
            subtech_x = subtechnique_coords[subtech_index][0]
            subtech_y = subtechnique_coords[subtech_index][1]
            net5.add_node(subtech, physics=False, label= d.attack.get_name_by_id(subtech), x=subtech_x, y=subtech_y, **d.attack.cfg.graph['subtechnique'])
            subtech_index += 1
            net5.add_edge(subtech, tech)
            
tactic_circles = generate_defense_circles(net5, 3900, 8, 100, 1, 100)




taken = []

# Add severity iteration so that highest severity rules are the closest

# for rule in s.rules:
#     for tech in rule.techniques:
#         tac = d.attack.find_tactic_in_hierarchical_index(tech)
        
#         tech_pos = (net5.get_node(tech)['x'],net5.get_node(tech)['y'])
#         circle = tactic_circles[tac]
#         closest_pos = d.find_closest_point(tech_pos, circle, taken)
#         if closest_pos:
#             net5.add_node(rule.id+tech, label=rule.title, color=d.attack.cfg.graph['severity_colors'][rule.severity], physics=False, x=closest_pos[0], y=closest_pos[1])
#             net5.add_edge(rule.id+tech, tech)
#             taken.append(closest_pos)       
        

net5.show('test5.html')

test5.html


In [40]:
len(tactic_circles['execution'])

126

{'color': '#C00',
 'physics': False,
 'x': 20000,
 'y': 10000,
 'mass': 100,
 'size': 100,
 'id': 'execution',
 'label': 'Execution',
 'shape': 'dot',
 'font': {'color': '#FFF'}}

In [12]:
for tac in d.attack.hierarchical_index:
    print(tac)

reconnaissance
resource-development
initial-access
execution
persistence
privilege-escalation
defense-evasion
credential-access
discovery
lateral-movement
collection
command-and-control
exfiltration
impact
